In [21]:
import numpy as np
import pandas as pd

In [22]:
coords = pd.read_csv('coords.csv')
# coords['x'] = coords['x']/2048
# coords['y'] = coords['y']/1360

coords

,ponto,x,y,X,Y,Z
0,1,860,833,0,0,0
1,2,592,712,0,0,1
2,3,1157,765,1,0,0
3,4,844,1196,0,1,0
4,5,581,1060,0,1,1
5,6,1137,1128,1,1,0
6,7,832,658,1,0,1
7,8,804,1015,1,1,1
8,9,347,601,0,0,2


In [24]:
pixels_coords = np.matrix(coords[['x','y']].values).T
pixels_coords_hom = np.vstack([pixels_coords, np.ones((1,len(coords)), int)])
pixels_coords_hom

matrix([[ 860,  592, 1157,  844,  581, 1137,  832,  804,  347],
        [ 833,  712,  765, 1196, 1060, 1128,  658, 1015,  601],
        [   1,    1,    1,    1,    1,    1,    1,    1,    1]],
       dtype=int64)

In [25]:
real_coords = np.matrix(coords[['X','Y','Z']].values).T
real_coords_hom = np.vstack([real_coords, np.ones((1,len(coords)), int)])
real_coords_hom

matrix([[0, 0, 1, 0, 0, 1, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 0, 1, 0],
        [0, 1, 0, 0, 1, 0, 1, 1, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)

In [26]:
rows = []
x = []
for i, (real_c, img_c) in enumerate(zip(real_coords_hom.T, pixels_coords_hom.T)):
    rows.append(np.concatenate([
        real_c, 
        np.array([[0,0,0,0]]),
        -img_c[0,0] * real_c
        ] , axis = 1))
    rows.append(np.concatenate([
        np.array([[0,0,0,0]]),
        real_c, 
        -img_c[0,1] * real_c
        ] , axis = 1))
    x.append(img_c[0,0])
    x.append(img_c[0,1])
    print(img_c[0,0], img_c[0,1])

860 833
592 712
1157 765
844 1196
581 1060
1137 1128
832 658
804 1015
347 601


In [32]:
A = np.concatenate(rows)

In [36]:
svd(A)[2][-1,:]

array([-1.63046237e-01,  1.34910251e-02,  1.86032745e-01, -6.66868143e-01,
        8.38350331e-02, -2.79157052e-01,  5.58468127e-02, -6.37052732e-01,
        4.43628497e-05, -2.18505110e-06, -4.63592915e-05, -7.67109901e-04])

In [28]:
def solution(U):
    # find the eigenvalues and eigenvector of U(transpose).U
    e_vals, e_vecs = np.linalg.eig(np.dot(U.T, U))  
    # extract the eigenvector (column) associated with the minimum eigenvalue
    return e_vecs[:, np.argmin(e_vals)] 

In [37]:
A.shape

(18, 12)

In [38]:
M = np.concatenate( [np.linalg.solve(A[:11, :11], x[:11]), np.array([1])] ).reshape(3,4)
lbd = np.sqrt(sum(np.array(M[2,:3]).reshape(-1) ** 2))
M= M/abs(lbd)
M= np.matrix(M)

In [43]:
M/M[2,3]

matrix([[ 3.52595579e+02, -1.70952483e+00, -2.38565470e+02,
          8.60000000e+02],
        [-3.12406069e+01,  3.83250484e+02, -8.55990110e+01,
          8.33000000e+02],
        [ 4.80514943e-02,  1.69318426e-02,  4.97204902e-02,
          1.00000000e+00]])

In [39]:
- M[:,:3]**-1 @ M[:, 3]

matrix([[-9.01126652],
        [-5.06948756],
        [-9.67728617]])

In [40]:
q1 = M[0, :3].T
q2 = M[1, :3].T
q3 = M[2, :3].T
q4 = M[:3, 3]

In [12]:
Ox = (q1.T @ q3)[0, 0]
Oy = (q2.T @ q3)[0, 0]


In [44]:
Ox, Oy

(996.9308715878755, 144.43347048187977)

In [14]:
# ABS é gambi
fx = ((q1.T@q1)[0,0] - Ox**2)** (1/2)
fy = ((q2.T@q2)[0, 0] - Oy**2) ** (1/2)

fx, fy

(5896.564233186125, 5531.814213519195)

In [15]:
cm = np.zeros((3,4))

In [16]:
cm[2, :] = M[2, :]
cm[0, 0] = (Ox * M[2,0] - M[0,0]) / fx
cm[0, 1] = (Ox * M[2,1] - M[0,1]) / fx
cm[0, 2] = (Ox * M[2,2] - M[0,2]) / fx
cm[1, 0] = (Oy * M[2,0] - M[1,0]) / fy
cm[1, 1] = (Oy * M[2,1] - M[1,1]) / fy
cm[1, 2] = (Oy * M[2,2] - M[1,2]) / fy

cm[0, 3] = (Ox * M[2,3] - M[0,3]) / fx
cm[1, 3] = (Oy * M[2,3] - M[1,3]) / fy


In [17]:
cm

array([[-0.72586081, -0.04428517,  0.68641453,  0.32620777],
       [ 0.09695495,  0.9670011 ,  0.23560267, -1.74851869],
       [ 0.67499236, -0.23784618,  0.69843719, 14.04727087]])

In [18]:
cam_matrix = np.vstack([-cm, np.array([0, 0, 0, 1])])

In [20]:
np.array(cam_matrix).T.reshape(-1)

array([  0.72586081,  -0.09695495,  -0.67499236,   0.        ,
         0.04428517,  -0.9670011 ,   0.23784618,   0.        ,
        -0.68641453,  -0.23560267,  -0.69843719,   0.        ,
        -0.32620777,   1.74851869, -14.04727087,   1.        ])